In [37]:
# # Active Nematics Simulations Test

from __future__ import division
import hoomd
import hoomd.md

#-----Define relevant variables

p_max = 1.0;
t_max = 6.0;
copies = 1;
init_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_init.gsd"

#-----Coupling Constants

tau1 = 10.0

#-----Define a simulation context

hoomd.context.initialize("");

#-----Extract the configuration of the system and expand the system

snap = hoomd.data.gsd_snapshot(init_file, frame = -1);
snap.replicate(copies,copies,copies);
system = hoomd.init.read_snapshot(snap);

notice(2): Group "all" created containing 1000 particles


In [38]:
for i in range(int(len(system.particles)-5)):
    del system.particles[1]

In [39]:
#-----Define each mesogen in the local reference frame of each center of mass

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
               types = ['A']*8,
               positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                            (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declare molecules as rigid bodies

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree();
lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl);
lj.set_params(mode = 'shift')

#------Define the interaction

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' , 'M', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' , 'A', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' , 'A', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP', 'M', epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP', 'A', epsilon = 1.0, sigma = 3.0);

#------Define activity parameters

all = hoomd.group.all();
N = len(all);

import numpy as np
activity = [(((np.random.rand(1)[0]-0.5)*6.0),
            ((np.random.rand(1)[0]-0.5)*6.0),
             0)
           for i in range(N)];

hoomd.md.force.active(group = all,
                      seed  = 137,
                      f_lst = activity,
                      rotation_diff = 0.005,
                      orientation_link = False);


#------Select an standar integrator

hoomd.md.integrate.mode_standard(dt = 0.005);

#-----Define some groups and make their union

nanoparticles = hoomd.group.type(name = 'NPs', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = 'NP_Mes', a = nanoparticles, b = mesogens);

#-----Integrate using NVT

npt = hoomd.md. integrate.nvt(group = groupNP_mes, kT = t_max, tau = tau1);

#-----Save data

log_file = "AN_T_" + str(t_max) + "_P_" + str(p_max) + "_ActivityON_03.log"
gsd_file = "AN_T_" + str(t_max) + "_P_" + str(p_max) + "_ActivityON_03.gsd"
meso_gsd_file = "AN_T_CM_" + str(t_max) + "_P_" + str(p_max) + "_ActivityON_03.gsd"

log = hoomd.analyze.log(filename = log_file,
                       quantities = ['num_particles', 
                                    'ndof',
                                    'translational_ndof',
                                    'rotational_ndof',
                                    'potential_energy',
                                    'kinetic_energy',
                                    'translational_kinetic_energy',
                                    'rotational_kinetic_energy',
                                    'temperature',
                                    'pressure',
                                    'volume'],
                       period = 1e2,
                       overwrite = True);
gsd = hoomd.dump.gsd(gsd_file, period = 1e2, group = hoomd.group.all(), overwrite = True);
meso_gsd = hoomd.dump.gsd(meso_gsd_file, period = 1e2, group = groupNP_mes, overwrite = True);

#-----Run the simulation

hoomd.run(5e4)

notice(2): constrain.rigid(): Creating 5 rigid bodies (adding 40 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
* M P Howard, A Statt, F Madutsa, T M Truskett, and A Z Panagiotopoulos.
  "Quantized bounding volume hierarchies for neighbor search in molecular
  simulations on graphics processing units", Computational Materials Science 164
  (2019) 139--146
-----
notice(2): Group "NPs" created containing 0 particles
notice(2): Group "rigid_center" created containing 5 particles
notice(2): Group "NP_Mes" created containing 5 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 45
notice(2): Neighbors included by diameter          : no
notice(2):